# Mapping Yelp Search Results - Part 1

- 04/26/22


## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [2]:
!pip install yelpapi

In [4]:
!pip install tqdm

     -------------------------------------- 78.4/78.4 KB 725.6 kB/s eta 0:00:00


In [5]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [6]:
# Load API Credentials
with open('C:/Users/ramos/.secret/yelp_api.json','r') as f:
    login = json.load(f)
login.keys() # put .keys() to hide your credentials

dict_keys(['client-id', 'api-key'])

In [7]:
# Instantiate YelpAPI Variable
yelp = YelpAPI(login['api-key'], timeout_s=5)
yelp

In [8]:
help(yelp)

Help on YelpAPI in module yelpapi.yelpapi object:

class YelpAPI(builtins.object)
 |  YelpAPI(api_key, timeout_s=None)
 |  
 |  This class implements the complete Yelp Fusion API. It offers access to the following APIs:
 |  
 |      * Autocomplete API - https://www.yelp.com/developers/documentation/v3/autocomplete
 |      * Business API - https://www.yelp.com/developers/documentation/v3/business
 |      * Business Match API - https://www.yelp.com/developers/documentation/v3/business_match
 |      * Event Lookup API - https://www.yelp.com/developers/documentation/v3/event
 |      * Event Search API - https://www.yelp.com/developers/documentation/v3/event_search
 |      * Featured Event API - https://www.yelp.com/developers/documentation/v3/featured_event
 |      * Phone Search API - https://www.yelp.com/developers/documentation/v3/business_search_phone
 |      * Reviews API - https://www.yelp.com/developers/documentation/v3/business_reviews
 |      * Search API - https://www.yelp.com/de

### Define Search Terms and File Paths

In [10]:
# set our API call parameters and filename before the first call
LOCATION = "Seattle, WA, 98102"
TERM = "Sushi"

In [13]:
LOCATION.split(',')[0]

'Seattle'

In [14]:
## Specify fodler for saving data
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)

# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = f"{FOLDER}{TERM}_{LOCATION.split(',')[0]}.json"
JSON_FILE

'Data/Sushi_Seattle.json'

### Check if Json File exists and Create it if it doesn't

In [16]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:  
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    with open(JSON_FILE,'w') as f:
        json.dump([],f)
    
    ## save the first page of results

        
## If it exists, inform user
else:
        print(f"[i] {JSON_FILE} already exists.")

[i] Data/Sushi_Seattle.json not found. Saving empty list to file.


### Load JSON FIle and account for previous results

In [17]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
        previous_results = json.load(f)
## set offset based on previous results
n_results = len(previous_results)
print(f' - {n_results} previous results found.')

 - 0 previous results found.


### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [19]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp.search_query(term=TERM,location=LOCATION)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [20]:
## How many results total?
results['total']

261

- Where is the actual data we want to save?

In [21]:
type(results['businesses'])

list

In [22]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [23]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil(results['total'] / results_per_page)
n_pages

14

In [26]:
for i in tqdm_notebook( range(0,n_pages+1)): # must do +1 to get all 20 pages
    ## The block of code we want to TRY to run
    try:
        time.sleep(.2)
        ## Read in results in progress file and check the length
        ## Load previous results and use len of results for offset
        with open(JSON_FILE,'r') as f:
            previous_results = json.load(f)    
        
        ## save number of results for to use as offset
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp.search_query(term=TERM,location=LOCATION,offset=n_results)

        ## append new results and save to file
        previous_results.extend(results['businesses'])

        with open(JSON_FILE,'w') as f:
            json.dump(previous_results,f)
    ## What to do if we get an error/exception.
    except Exception as e:
        print(e)
        break


  0%|          | 0/15 [00:00<?, ?it/s]

## Open the Final JSON File with Pandas

In [27]:
df = pd.read_json(JSON_FILE)
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,YSQiqH7RIWORk_Qp-A4SOg,shiros-seattle,Shiro's,https://s3-media3.fl.yelpcdn.com/bphoto/4LNql5...,False,https://www.yelp.com/biz/shiros-seattle?adjust...,2363,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 47.6147782162181, 'longitude': -1...","[pickup, delivery]",$$$$,"{'address1': '2401 2nd Ave', 'address2': '', '...",+12064439844,(206) 443-9844,2916.810435
1,tg9Fsdky2V-Bg3xiSIT4sg,taneda-sushi-in-kaiseki-seattle-2,Taneda Sushi in Kaiseki,https://s3-media3.fl.yelpcdn.com/bphoto/-_C6o6...,False,https://www.yelp.com/biz/taneda-sushi-in-kaise...,222,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",5.0,"{'latitude': 47.62063, 'longitude': -122.32134}",[],$$$$,"{'address1': '219 Broadway E', 'address2': 'St...",,,1702.642492
2,1kltyyJOETOpxkFa1sBdgQ,aoki-sushi-and-grill-seattle-5,Aoki Sushi & Grill,https://s3-media4.fl.yelpcdn.com/bphoto/cP81nh...,False,https://www.yelp.com/biz/aoki-sushi-and-grill-...,70,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 47.6249214, 'longitude': -122.321...",[delivery],NaN,"{'address1': '621 Broadway E', 'address2': '',...",+12065574595,(206) 557-4595,1228.374005
3,vs5dhED5Yf8kqPp06OD2QQ,momiji-seattle,Momiji,https://s3-media2.fl.yelpcdn.com/bphoto/6FKB4E...,False,https://www.yelp.com/biz/momiji-seattle?adjust...,1590,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 47.61482, 'longitude': -122.31657}",[delivery],$$,"{'address1': '1522 12th Ave', 'address2': '', ...",+12064574068,(206) 457-4068,2401.274089
4,jAgEaQUQMvGC2PsxYjBWOQ,katsu-ya-seattle-seattle,Katsu-ya Seattle,https://s3-media3.fl.yelpcdn.com/bphoto/TepKrk...,False,https://www.yelp.com/biz/katsu-ya-seattle-seat...,140,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 47.619252, 'longitude': -122.338099}",[delivery],$$,"{'address1': '122 Westlake Ave N', 'address2':...",+12065800080,(206) 580-0080,2118.369711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,NaxujkZqT7kixb-XmJcHZg,zoka-coffee-roaster-and-tea-company-seattle-5,Zoka Coffee Roaster & Tea Company,https://s3-media4.fl.yelpcdn.com/bphoto/4xpDqE...,False,https://www.yelp.com/biz/zoka-coffee-roaster-a...,380,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.0,"{'latitude': 47.6688199, 'longitude': -122.33287}",[delivery],$$,"{'address1': '2200 N 56th St', 'address2': '',...",+12065454277,(206) 545-4277,3735.147088
257,mOfmcAg_c4d72YEmz_rkEQ,chew-chews-and-eatery-seattle,Chew Chews & Eatery,https://s3-media3.fl.yelpcdn.com/bphoto/_DmNnO...,False,https://www.yelp.com/biz/chew-chews-and-eatery...,15,"[{'alias': 'newamerican', 'title': 'American (...",3.5,"{'latitude': 47.6049072432661, 'longitude': -1...","[delivery, pickup]",$,"{'address1': '700 5th Ave', 'address2': 'Ste 6...",+12062230820,(206) 223-0820,3458.414141
258,NGJ08D3NgxUR-DnUQB7mmA,bambinos-pizzeria-seattle,Bambino's Pizzeria,https://s3-media3.fl.yelpcdn.com/bphoto/ELDSUV...,False,https://www.yelp.com/biz/bambinos-pizzeria-sea...,535,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 47.6179, 'longitude': -122.34761}","[delivery, pickup]",$$,"{'address1': '401 Cedar St', 'address2': None,...",+12062692222,(206) 269-2222,2673.472893
259,tApJIRcvY0ARGftZyKdpeA,qfc-seattle-40,QFC,https://s3-media2.fl.yelpcdn.com/bphoto/XnBarc...,False,https://www.yelp.com/biz/qfc-seattle-40?adjust...,122,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.0,"{'latitude': 47.6613353937864, 'longitude': -1...",[],$$,"{'address1': '1801 N 45th Street', 'address2':...",+12066329253,(206) 632-9253,3033.026309


In [28]:
df.duplicated(subset='id').sum()

0

In [29]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Sushi_Seattle.csv.gz'

In [30]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression='gzip',index=False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [31]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 253,303 Bytes
CSV.GZ FILE: 36,047 Bytes
the csv.gz is 7.02702027908009 times smaller!


## Next Class: Processing the Results and Mapping 